In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from vits.utils.utils import load_wav_to_torch
from vits.utils.mel_processing import spectrogram_torch

from vits.model import commons
from vits.utils import utils
from vits.model.models import SynthesizerTrn
from vits.text.symbols import symbols
from vits.text import cleaned_text_to_sequence, text_to_sequence, batch_text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps, language_code):
    text_norm = text_to_sequence(text, str(language_code))
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0) 
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [3]:
# device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

In [4]:
hps = utils.get_hparams_from_file("vits/configs/vits_base.json")

CUDA_LAUNCH_BLOCKING=1
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=61,
    **hps.model).to(device)
_ = net_g.eval()

# _ = utils.load_checkpoint("vits/checkpoints/vits_pl_test/G_?.pth", net_g, None)

/home/ubuntu/miniforge3/envs/vits_pl/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [7]:
def dataCollate(txt_lists):
    max_text_len = max([len(x) for x in txt_lists])
    text_lengths = torch.LongTensor(len(txt_lists))
    text_padded = torch.LongTensor(len(txt_lists), max_text_len)
    text_padded.zero_()

    for i in range(len(txt_lists)):
        text = torch.LongTensor(txt_lists[i])
        text_padded[i, :len(text)] = text
        text_lengths[i] = len(text)

    return text_padded, text_lengths

def find_mask(audio):
    mask = 0
    for i in range(len(audio)-1, 0, -1):
        if torch.abs(audio[i]) < 0.01 and torch.mean(torch.abs(audio[i-100:i])) < 0.001:
            mask = i
            break
    
    return mask

def concat_audio(audio_list):
    final_audio = []

    for audio in audio_list:
        mask = find_mask(audio[0].data)
        final_audio.append(audio[0].data[:mask])

    final_audio = torch.cat(final_audio, dim=0)
    return final_audio

language_code = 0
group_size = 5
input_text = "반갑습니다. 당신의 상담사로 이곳에 오게 되었습니다. 당신의 불편한 점을 말씀해주시겠어요?"
txt_lists = batch_text_to_sequence(input_text, str(language_code), group_size)
text_padded, text_lengths = dataCollate(txt_lists)

sid=None
lid=None

with torch.no_grad():

    x_tst = text_padded.to(device)
    x_tst_lengths = text_lengths.to(device)

    sid = torch.LongTensor([0]).to(device)
#     audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, lid=lid, noise_scale=0.333, noise_scale_w=0.1, length_scale=1.2)

# final_audio = concat_audio(audio[0]).cpu().float().numpy()
# ipd.display(ipd.Audio(final_audio, rate=hps.data.sampling_rate, normalize=False))

# from scipy.io.wavfile import write
# write("/workspace/test.wav", 22050, final_audio)